In [2]:
# !pip install osrm-py
!pip install openrouteservice

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
import pandas as pd
import numpy as np
import json
import requests
import openrouteservice

In [2]:
# import the data
df = pd.read_excel('..\\..\\data\\17.03.31 Hackathon Revised Randomized Location Data v2F.xlsx', sheet_name = 'Student Information', header=2)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22422 entries, 0 to 22421
Data columns (total 17 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Unnamed: 0                               0 non-null      float64
 1   Street Number                            22420 non-null  object 
 2   Street Name                              22420 non-null  object 
 3   Zip Code                                 22420 non-null  float64
 4   Latitude                                 22420 non-null  float64
 5   Longitude                                22420 non-null  float64
 6   Pickup Type                              22420 non-null  object 
 7   Grade                                    22421 non-null  object 
 8   Geocode                                  22420 non-null  float64
 9   Neighborhood Safety Score                22420 non-null  float64
 10  Proposed Maximium Walk to Stop Distance  22420

In [4]:
df.head()

,Unnamed: 0,Street Number,Street Name,Zip Code,Latitude,Longitude,Pickup Type,Grade,Geocode,Neighborhood Safety Score,Proposed Maximium Walk to Stop Distance,Assigned School,Current School Start Time,Current School End Time,School Address,School Latitude,School Longitude
0,NaN,147,Kelton St,2134.0,-71.13973,42.34797,Corner,3,2.502500e+10,3.0,0.5,Marty Barrett School,09:30:00,16:10:00,50 Beechcroft Street,-71.161501,42.351079
1,NaN,2,Barstow St,2134.0,-71.14253,42.35302,Corner,2,2.502500e+10,5.0,0.4,Marty Barrett School,09:30:00,16:10:00,50 Beechcroft Street,-71.161501,42.351079
2,NaN,45,Saunders St,2134.0,-71.14195,42.35347,Corner,3,2.502500e+10,5.0,0.5,Marty Barrett School,09:30:00,16:10:00,50 Beechcroft Street,-71.161501,42.351079
3,NaN,630,Cambridge St,2134.0,-71.14337,42.35132,Corner,1,2.502500e+10,6.0,0.2,Marty Barrett School,09:30:00,16:10:00,50 Beechcroft Street,-71.161501,42.351079
4,NaN,575,Cambridge St,2134.0,-71.14103,42.35257,Corner,5,2.502500e+10,5.0,0.5,Marty Barrett School,09:30:00,16:10:00,50 Beechcroft Street,-71.161501,42.351079


In [5]:
# each school have the most students?
count_schools = pd.DataFrame(df.groupby('Assigned School').size().sort_values(ascending=True)) # John L.  Taylor School

In [12]:
count_schools[:30]

,0
Assigned School,
Ben Mondor School,15
Hanley Ramirez School,17
Steven Wright School,20
Tom Yawkey School,21
Joe Kelly School,23
Jackie Bradley Jr. School,26
Steve Selsky School,30
Josh Rutledge School,33
Wes Ferrell School,34


In [7]:
# keep only school with most students
df_williams = df.copy()
df_williams = df_williams.loc[df_williams['Assigned School']=='Williams Jerez School']

In [8]:
len(df_williams)

50

In [9]:
df_williams.drop('Unnamed: 0', axis=1, inplace=True)
df_williams.reset_index(inplace=True)

In [10]:
df_williams.iloc[49]

index                                                      21480
Street Number                                                 20
Street Name                                          Lagrange St
Zip Code                                                  2132.0
Latitude                                              -71.144545
Longitude                                               42.27356
Pickup Type                                               Corner
Grade                                                          6
Geocode                                            25025130300.0
Neighborhood Safety Score                                    3.0
Proposed Maximium Walk to Stop Distance                      0.5
Assigned School                            Williams Jerez School
Current School Start Time                               09:30:00
Current School End Time                                 17:00:00
School Address                                  200 Heath Street
School Latitude          

In [11]:
# duplicated locations?
df_williams.duplicated(subset=['Longitude','Latitude']).sum()

3

In [13]:
# remove duplicated records
df_williams.drop_duplicates(subset=['Longitude','Latitude'], inplace=True)

In [14]:
len(df_williams)

47

In [16]:
df_williams.reset_index(inplace=True)

In [18]:
# create a list of locations
locations = []
for i in range(len(df_williams)):
    if i == 0:
        locations.append((df_williams['School Latitude'][i],df_williams['School Longitude'][i])) # adds the location of the school
        locations.append((df_williams['Latitude'][i],df_williams['Longitude'][i])) # adds the location of the first student
    else:
        locations.append((df_williams['Latitude'][i],df_williams['Longitude'][i])) # adds the location of all the other

In [19]:
len(locations)

48

In [20]:
locations

[(-71.10693577, 42.32629276),
 (-71.08972, 42.3086),
 (-71.09174, 42.31168),
 (-71.09214, 42.31111),
 (-71.12167, 42.28631),
 (-71.1149, 42.28767),
 (-71.07957, 42.35197),
 (-71.08296, 42.35237),
 (-71.07729, 42.35074),
 (-71.090942, 42.327921),
 (-71.09115, 42.32668),
 (-71.09077, 42.32064),
 (-71.09033, 42.32577),
 (-71.092242, 42.324232),
 (-71.09133, 42.329295),
 (-71.0919, 42.32102),
 (-71.09207, 42.33004),
 (-71.09237, 42.31752),
 (-71.092202, 42.32125),
 (-71.09312, 42.34939),
 (-71.0844, 42.33535),
 (-71.089062, 42.335189),
 (-71.08473, 42.33712),
 (-71.0587, 42.3594),
 (-71.08427, 42.33578),
 (-71.08478, 42.33794),
 (-71.08439, 42.33733),
 (-71.0845, 42.3354),
 (-71.083765, 42.335812),
 (-71.0863, 42.33704),
 (-71.08349, 42.34207),
 (-71.08061, 42.34293),
 (-71.081, 42.34271),
 (-71.083734, 42.338159),
 (-71.12095, 42.3109),
 (-71.11726, 42.2936),
 (-71.13231, 42.30399),
 (-71.12239, 42.32172),
 (-71.12959, 42.30514),
 (-71.11836, 42.29041),
 (-71.13266, 42.30302),
 (-71.12564

In [21]:
# https://openrouteservice-py.readthedocs.io/en/latest/
# https://github.com/GIScience/openrouteservice-py/blob/master/examples/basic_example.ipynb

def get_driving_distance_matrix(coordinates):
    
    """ Function receives a list of locations coordinates and return a matrix with distances (m) and duration (s)"""
    
    client = openrouteservice.Client(key='5b3ce3597851110001cf6248221897b119124952acf8240fb50696f9')
    #call API
    matrix = client.distance_matrix(
        locations=coordinates,
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
    )
    
    return matrix

In [22]:
def call_api(locations):
    """ Function receives a list of locations as saves the output in a json file """
    distance_matrix = get_driving_distance_matrix(locations)
    # save json file
    with open('distance_matrix.json', 'w') as outfile:
        json.dump(distance_matrix, outfile)

In [23]:
# save json file with the matrix of distances
call_api(locations)

In [24]:
def read_jsons(file):
    with open(file) as json_file:
        data = json.load(json_file)
    return data

In [26]:
distance_matrix=read_jsons('distance_matrix.json')['distances']
duration_matrix=read_jsons('distance_matrix.json')['durations']

In [31]:
print(distance_matrix[0])

[0.0, 3607.19, 2602.87, 2714.14, 6050.23, 5729.21, 5049.18, 5226.9, 5347.26, 1944.1, 1740.1, 1880.86, 2031.62, 1945.9, 1819.27, 1766.3, 1678.52, 2279.79, 1715.8, 4166.54, 3378.59, 2683.18, 2671.92, 6470.33, 3093.06, 2793.82, 2720.01, 3388.51, 3061.59, 2667.19, 3698.18, 3758.04, 3593.39, 2904.68, 3106.7, 5086.47, 4667.54, 2711.5, 4446.14, 5170.06, 4737.56, 3465.22, 5037.69, 4416.36, 3149.37, 6807.08, 5986.27, 8148.72]


In [32]:
print(duration_matrix[0])

[0.0, 382.92, 281.77, 287.84, 578.0, 538.86, 553.49, 609.08, 590.39, 231.06, 194.34, 230.04, 246.81, 245.49, 201.79, 211.24, 180.22, 295.39, 200.33, 478.79, 395.24, 308.58, 281.15, 750.46, 349.52, 299.99, 286.71, 397.03, 343.85, 290.83, 384.28, 415.68, 388.75, 303.93, 358.93, 444.89, 509.58, 378.4, 486.48, 451.75, 522.18, 410.02, 441.7, 481.12, 342.08, 676.7, 629.98, 768.42]
